In [8]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import tqdm as tqdm
from diffusion_models import Weighted_Cascade, Trivalency_Model, Independent_Cascade, Decreasing_Cascade, linear_threshold, general_threshold
import pandas as pd


In [2]:
pb2020 = nx.read_gml('/Users/davidguzman/Documents/GitHub/Network-Science-final-Maximization-/Proud_Boys_Datsets/PB2020.gml')

In [3]:
#invert edge direction for PB2020
pb2020 = pb2020.reverse()

In [4]:
#number of nodes and edges
print("Number of nodes: ", pb2020.number_of_nodes())
print("Number of edges: ", pb2020.number_of_edges())

Number of nodes:  13217
Number of edges:  14477


In [5]:
weighted_cascade_model_2020 = Weighted_Cascade()
weighted_cascade_model_2020.fit(pb2020)

In [6]:
seed = ['principe_giovan',
 'Premises187',
 'MoralDK',
 'proudboy_',
 'enrique_tarrio',
 'GavinM_ProudBoy',
 'proudboy2012',
 'ProudBoysGBG',
 'ProudBoysUS',
 'ProudBoyzLondon',
 'DetroitProudBoy',
 'Father_Rhyme',
 'ColoradoPOYB',
 'Proudboy13',
 'realDonaldTrump',
 'PrisonPlanet',
 'TuckerCarlson',
 'Cernovich',
 'seanhannity',
 'IngrahamAngle',
 'benshapiro',
 'RealCandaceO',
 'IvankaTrump',
 'joerogan',
 'scrowder',
 'jordanbpeterson',
 'RealJamesWoods',
 'Lauren_Southern',
 'TheNotoriousMMA',
 'DonaldJTrumpJr']

In [17]:
#get the attributes of the nodes whose labels are in the seed
seed_nodes = [pb2020.nodes[node] for node in seed]
# for the nodes in seed_nodes, get all attributes and store them in a pandas dataframe
seed_nodes_df = pd.DataFrame(seed_nodes)
#add the labels of the nodes to the dataframe as the first column
seed_nodes_df.insert(0, 'label', seed)


In [20]:
#drop all columns after the statuses
seed_nodes_df = seed_nodes_df.iloc[:, :6]
#drop all rows that have a NaN value
seed_nodes_df = seed_nodes_df.dropna()


In [23]:
new_seed = seed_nodes_df['label'].tolist()

In [25]:
individual_influence = []
for label in new_seed:
    activate_nums = weighted_cascade_model_2020.monte_carlo_diffusion_all([label], num_simulations=10)
    individual_influence.append(sum(activate_nums))

In [27]:
seed_nodes_df['influence_wcm'] = individual_influence

In [28]:
seed_nodes_df

,label,faves,followers,friends,listed,statuses,influence_wcm
0,principe_giovan,8253.0,625.0,605.0,6.0,693.0,551.8
2,MoralDK,14609.0,216.0,322.0,5.0,6421.0,233.1
7,ProudBoysGBG,407.0,48.0,50.0,1.0,213.0,42.1
8,ProudBoysUS,604.0,3096.0,44.0,12.0,482.0,35.0
12,ColoradoPOYB,1.0,12.0,9.0,1.0,5.0,6.6
14,realDonaldTrump,5.0,86274420.0,50.0,119970.0,56355.0,1.0
15,PrisonPlanet,8403.0,1193784.0,754.0,7401.0,72065.0,1.0
16,TuckerCarlson,75.0,3971466.0,97.0,11406.0,4016.0,1.0
17,Cernovich,71847.0,721260.0,3176.0,4804.0,120132.0,1.0
18,seanhannity,110.0,5232793.0,6.0,19662.0,7727.0,1.0
